In [13]:
import numpy as np
import pickle
import torch
import sys
import os
import cyipopt as ipopt
import os
import time
'''
        minimize_y 1/2 * y^T Q y + p^Ty
        s.t.       1/2 * y^T D y = x
                   1/2 * y^T H y + G^T y <= h
                   L<= x <=U
'''

# num_var = 100
# num_ineq = 50
# num_eq = 30
# num_examples = 100

# np.random.seed(2023)
# Q = np.diag(np.random.rand(num_var)*0.5)
# p = np.random.uniform(-1, 1, num_var)
# X = np.random.uniform(-0.1, 0.1, size=(num_examples, num_eq))
# G = np.random.uniform(-1, 1, size=(num_ineq, num_var))
# h = np.random.uniform(0, 0.1, size=(num_ineq))
# H = np.random.uniform(-0.1, 0.1,  size=(num_ineq, num_var))
# # randomly set some of the elements in H to its negative value
# H = [np.diag(H[i]) for i in range(num_ineq)]
# H = np.array(H)
# L = np.ones((num_var)) * -5
# U = np.ones((num_var)) * 5
# A = np.random.uniform(-1, 1, size=(num_eq, num_var))

# data = {'Q':Q,
#         'p':p,
#         'X':X,
#         'G':G,
#         'H':H,
#         'h':h,
#         'L':L,
#         'U':U,
#         'A':A,
#         'Y':[]}

'\n        minimize_y 1/2 * y^T Q y + p^Ty\n        s.t.       1/2 * y^T D y = x\n                   1/2 * y^T H y + G^T y <= h\n                   L<= x <=U\n'

In [53]:
import numpy as np
import pickle
import torch

import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], os.pardir, os.pardir))
from utils import SimpleProblem

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

torch.set_default_dtype(torch.float64)

import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr, LinExpr

from utils import build_gurobi_model
import time
from qcqp_utils import QCQPProbem

from model_utils import NNSolver_eq_proj

filepath = "/home/jxxiong/A-xjx/DC3/datasets/convex_qcqp/random_convex_qcqp_dataset_var100_ineq50_eq30_ex10000"
with open(filepath, 'rb') as f:
    data = pickle.load(f)

with open(filepath, 'rb') as f:
    dataset = pickle.load(f)
data = QCQPProbem(dataset)

arg_path = "/home/jxxiong/A-xjx/DC3/results/QCQPProblem-100-50-30-10000/method_eq_proj/ec03c25d47020fa8cefa940700db9572d3dd7925/1713679980-1177857/args.dict"
args = pickle.load(open(arg_path, 'rb'))

model = NNSolver_eq_proj(data, args)

model_path = "/home/jxxiong/A-xjx/DC3/results/QCQPProblem-100-50-30-10000/method_eq_proj/ec03c25d47020fa8cefa940700db9572d3dd7925/1713679980-1177857/solver_net.dict"
model.load_state_dict(torch.load(model_path))

model.eval()
DEVICE = 'cpu'
model.to(DEVICE)

for attr in dir(data):
    var = getattr(data, attr)
    if not callable(var) and not attr.startswith("__") and torch.is_tensor(var):
        try:
            setattr(data, attr, var.to(DEVICE))
        except AttributeError:
            pass
data._device = DEVICE

In [54]:
Q = data.Q
p = data.p
X = data.X
G = data.G
H = data.H
h = data.h
L = data.L
U = data.U
A = data.A

Q_np = data.Q_np
p_np = data.p_np
X_np = data.X.detach().numpy()
G_np = data.G_np
H_np = data.H_np
h_np = data.h_np
L_np = data.L_np
U_np = data.U_np
A_np = data.A_np

num_var = A.shape[1]
num_eq = A.shape[0]
num_ineq = G.shape[0]
num_examples = X.shape[0]

idx = 9000

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [55]:
# def qcqp_ipopt(H, G, h):
    
#     def ineq_constraint(y):
#         return 0.5 * (y @ H @ y) + G@y - h
    
#     def ineq_jacobian(y):
#         return H@y + G
    
#     ineq_constraint_dict = {'type': 'ineq',
#                             'fun': ineq_constraint,
#                             'jac': ineq_jacobian}

#     return ineq_constraint_dict


# def eq_ipopt(A, x):
#     def eq_constraint(y):
#         return A@y - x
    
#     def eq_jacobian(y):
#         return A

#     eq_constraint_dict = {'type': 'eq',
#                             'fun': eq_constraint,
#                             'jac': eq_jacobian}
#     return eq_constraint_dict

# Y = []
# for n in range(num_examples):
#     Xi = X[n]
#     y0 = np.array([0.05]*num_var)
#     bounds = [(L[i], U[i]) for i in range(num_var)]
#     eq_constraints = []
#     ineq_constraints = []
#     eq_constraints.append(eq_ipopt(A, Xi))
    
#     for i in range(num_ineq):
#         Ht = H[i]
#         Gt = G[i]
#         ht = h[i]
#         ineq_con = qcqp_ipopt(Ht, Gt, ht)
#         ineq_constraints.append(ineq_con)
    
#     def objective(y):
#         return 0.5 * (y @ Q @ y) + p@y

#     def grad(y):
#         return Q@y + p
#     start_time = time.time()
#     result = ipopt.minimize_ipopt(objective, y0, jac=grad, bounds=bounds, constraints=ineq_constraints+eq_constraints)
#     print('status:', result.success)
#     print(time.time() - start_time)
#     # if result.success:
#     #     Y.append(result.x)
#     #     print(n)

In [56]:
# result

In [57]:
# y = result.x

In [58]:
# res = []
# for i in range(num_ineq):
#     Ht = H[i]
#     Gt = G[i]
#     ht = h[i]
#     res.append(0.5 * y @ Ht @ y + Gt@y -ht)
# res

In [59]:
# import numpy as np
# from cyipopt import minimize_ipopt

# # Define the matrices and vectors for demonstration purposes
# n = 3  # Dimension of y
# Q = np.array([[1, 0, 0], [0, 2, 0], [0, 0, 3]])
# H1 = np.array([[2, 0, 0], [0, 1, 0], [0, 0, 1]])
# H2 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 2]])
# p = np.array([1, 1, 1])
# g1 = np.array([-1, -2, -3])
# g2 = np.array([2, 1, -1])
# h1 = 1
# h2 = 0.5

# # Objective function and its gradient
# def objective(y):
#     return y.T @ Q @ y + p.T @ y

# def objective_grad(y):
#     return 2 * Q @ y + p

# # Constraint functions and Jacobians
# def constraint1(y):
#     return np.array([y.T @ H1 @ y + g1.T @ y - h1])

# def constraint1_jac(y):
#     return np.array([2 * H1 @ y + g1]).reshape(1, -1)

# def constraint2(y):
#     return np.array([y.T @ H2 @ y + g2.T @ y - h2])

# def constraint2_jac(y):
#     return np.array([2 * H2 @ y + g2]).reshape(1, -1)

# # Define constraints in a tuple format compatible with CyIpopt
# constraints = [{'type': 'ineq',
#                 'fun': constraint1,
#                 'jac': constraint1_jac},
#                {'type': 'ineq',
#                 'fun': constraint2,
#                 'jac': constraint2_jac}]

# # Initial guess
# y0 = np.array([0.1, 0.1, 0.1])

# # Solve the problem
# result = minimize_ipopt(objective, y0, jac=objective_grad, constraints=constraints, options={'tol': 1e-6, 'acceptable_tol': 1e-5, 'max_iter': 1000, 'disp': True})

# print("Solution:", result.x)
# print("Objective at solution:", result.fun)


In [60]:
# y = result.x
# y.T @ H1 @ y + g1.T @ y - h1

In [61]:
# result.info['g']

In [62]:
class nonconvex_ipopt(object):
    def __init__(self, Q, p, A, H, G):
        self.Q = Q
        self.p = p
        self.A = A
        self.H = H
        self.G = G

    def objective(self, y):
        return 0.5*(y.T @ self.Q @ y) + self.p.T@y

    def gradient(self, y):
        return self.Q@y + self.p

    def constraints(self, y):
        constrs = [self.A@y]
        for i in range(self.H.shape[0]):
            constrs.append(0.5*(y.T@self.H[i]@y) + self.G[i].T@y)
        return np.hstack(constrs)

    def jacobian(self, y):
        jac = [self.A.flatten()]
        for i in range(self.H.shape[0]):
            jac.append((self.H[i]@y + self.G[i]).flatten())
        return np.concatenate(jac)

def build_ipopt(Q, p, A, Xi, H, G, h, lb=None, ub=None, y0=None, solver_type='ipopt', tol=1e-8):
    # if y0 is None:
    #     y0 = np.linalg.pinv(A)@Xi  # feasible initial point
    num_var = A.shape[1]
    # upper and lower bounds on variables
    if lb is None:
        lb = -np.infty * np.ones(num_var)
    if ub is None:
        ub = np.infty * np.ones(num_var)

    # upper and lower bounds on constraints
    cl = np.hstack([Xi, -np.inf * np.ones(G.shape[0])])
    cu = np.hstack([Xi, h])

    nlp = ipopt.problem(n=num_var,
                        m=len(cl),
                        problem_obj=nonconvex_ipopt(Q, p, A, H, G),
                        lb=lb,
                        ub=ub,
                        cl=cl,
                        cu=cu)

    nlp.add_option('tol', tol)
    nlp.add_option('print_level', 0) # 3)

    # start_time = time.time()
    # y, info = nlp.solve(y0)
    # end_time = time.time()
    return nlp

In [63]:
nlp = build_ipopt(Q_np, p_np, A_np, X_np[idx], H_np, G_np, h_np, L_np, U_np, y0 = np.array([0.05]*num_var))

/tmp/ipykernel_3160379/560160664.py:41: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


In [64]:
start_time = time.time()
y, info = nlp.solve(np.array([0.05]*num_var))
print(time.time() - start_time)

0.0862724781036377


In [65]:
info

{'x': array([-3.36131055e+00,  3.67792238e-01,  9.87953030e-01, -2.45765613e+00,
        -3.80188991e-01,  3.01940863e+00, -6.86498460e-01, -4.19053667e-01,
        -1.74249465e+00, -1.27290879e+00,  6.22008068e-01, -1.62076736e-01,
        -3.29603765e-01,  3.26903699e-02,  1.20462596e+00, -5.00000005e+00,
         5.48106576e-01, -1.40127467e+00,  4.71232234e+00,  7.05604966e-01,
        -5.35574953e-01,  1.73337298e+00,  2.98514860e+00,  2.86370758e-01,
        -5.00000005e+00, -3.16241167e+00,  2.16876497e+00, -4.02751108e+00,
         3.25951377e+00, -6.74445226e-01, -2.45219233e-02,  7.33418467e-01,
         1.13033550e+00,  1.98029759e+00, -2.95847985e+00, -1.01089736e+00,
         1.90671017e+00,  3.08109378e+00, -9.74830216e-01,  8.14520842e-01,
        -1.59903703e+00, -7.56995750e-01,  4.67227946e-01,  1.62171606e+00,
         5.00000005e+00,  1.60731213e+00, -6.37380234e-01, -2.39101269e+00,
         1.85225270e+00, -4.28265480e+00,  2.83870120e+00,  1.06419273e+00,
       

In [66]:
info['obj_val']

-39.8476382080661

In [67]:
res = []
for i in range(num_ineq):
    res.append(0.5 * y.T @ H_np[i] @ y + G_np[i].T @ y - h_np[i])
res = np.array(res)
print(res.max())
active_ineq = np.where(np.abs(res) < 1e-4)[0]
active_ineq

9.966808572903219e-09


array([ 2,  3,  6, 10, 11, 14, 15, 16, 18, 19, 21, 23, 24, 25, 26, 27, 28,
       29, 31, 36, 37, 40, 42, 43, 44])

In [68]:
G_remaining = G_np[active_ineq]
H_remaining = H_np[active_ineq]
h_remaining = h_np[active_ineq]

nlp_r = build_ipopt(Q_np, p_np, A_np, X_np[idx], H_remaining, G_remaining, h_remaining, L_np, U_np, y0 = np.array([0.5]*num_var))
start_time = time.time()
# y_r, info_r = nlp_r.solve(np.array([0.5]*num_var))
y_r, info_r = nlp_r.solve(y)
print(time.time() - start_time)
info_r['obj_val']

0.05505108833312988


/tmp/ipykernel_3160379/560160664.py:41: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


-39.84763820806611

In [69]:
L_r = -np.infty * np.ones(num_var)
U_r = np.infty * np.ones(num_var)
active_L = np.where(np.abs(L_np - y) < 1e-4)[0]
active_U = np.where(np.abs(U_np - y) < 1e-4)[0]
L_r[active_L] = L_np[active_L]
U_r[active_U] = U_np[active_U]
print(active_L)
print(active_U)

[15 24]
[44]


In [70]:
nlp_r = build_ipopt(Q_np, p_np, A_np, X_np[0], H_remaining, G_remaining, h_remaining, L_r, U_r, y0 = np.array([0.5]*num_var))
start_time = time.time()
# y_r, info_r = nlp_r.solve(np.array([0.5]*num_var))
y_r, info_r = nlp_r.solve(y)
print(time.time() - start_time)
info_r['obj_val']

0.0449979305267334


/tmp/ipykernel_3160379/560160664.py:41: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


-39.82636601613135

In [78]:
x_input = torch.tensor(data.X[idx]).unsqueeze(0)
start_time = time.time()
Yhat = model(x_input)
Ystar = data.projection(x_input, Yhat)
end_time = time.time()
inference_time = end_time - start_time
print("Inference time: ", inference_time)
# Ystar = Ystar.detach().numpy().flatten()
# ineq_sorted_index = np.argsort(-G_np@Ystar + h_np)
# ineq_sorted = np.sort(-G_np@Ystar + h_np)
# print(ineq_sorted)
# print(ineq_sorted_index)
# inactive_ineq_index = ineq_sorted_index[ineq_sorted > 333]
# print(inactive_ineq_index)

Inference time:  0.00042366981506347656


/tmp/ipykernel_3160379/339485510.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_input = torch.tensor(data.X[idx]).unsqueeze(0)


In [79]:
y = Ystar.squeeze(0)
res = []
for i in range(num_ineq):
    res.append((0.5 * y.T @ data.H[i] @ y + data.G[i].T @ y - data.h[i]).detach().numpy())
res = np.array(res)
res

/tmp/ipykernel_3160379/552142522.py:4: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343997789/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  res.append((0.5 * y.T @ data.H[i] @ y + data.G[i].T @ y - data.h[i]).detach().numpy())


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [22]:
q = torch.matmul(torch.tensor(data.H), Ystar.T).permute(2, 0, 1)
q = (q * Ystar.view(Ystar.shape[0], 1, -1)).sum(-1)
res = 0.5 * q + torch.matmul(Ystar, torch.tensor(data.G).T) - torch.tensor(data.h)
res

/tmp/ipykernel_3078455/124799448.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  q = torch.matmul(torch.tensor(data.H), Ystar.T).permute(2, 0, 1)
/tmp/ipykernel_3078455/124799448.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  res = 0.5 * q + torch.matmul(Ystar, torch.tensor(data.G).T) - torch.tensor(data.h)


tensor([[ -6.0679,  -4.1733,  -0.2182,  -0.1012, -23.3111,  -0.6816,  -0.0509,
         -14.9890, -10.1695, -13.6509,  -0.1196,  -0.0728, -15.2542, -11.0072,
          -0.6862,  -0.1827,  -0.0970,  -6.1271,  -0.0838,  -0.2114,  -7.0946,
          -0.4256, -10.4887,  -1.0358,  -0.1518,  -0.0925,  -0.0693,  -0.2060,
          -0.2180,  -0.2592, -14.5927,  -0.1097, -13.4422,  -1.9290,  -1.7700,
         -11.4653,  -0.2895,  -0.1309,  -9.8097, -22.2818,  -0.0933, -12.9845,
          -1.9990,  -0.1097,  -0.1107,  -3.7809, -16.6783,  -4.2474,  -1.9142,
         -19.5440]], grad_fn=<SubBackward0>)

In [26]:
inactive_ineq = torch.where(res[0] < -3)[0].numpy()
inactive_ineq

array([ 0,  1,  4,  7,  8,  9, 12, 13, 17, 20, 22, 30, 32, 35, 38, 39, 41,
       45, 46, 47, 49])

In [34]:
remianing_ineq = torch.where(res[0] >= -3)[0].numpy()
G_remaining = G_np[remianing_ineq]
H_remaining = H_np[remianing_ineq]
h_remaining = h_np[remianing_ineq]

nlp_r = build_ipopt(Q_np, p_np, A_np, X_np[idx], H_remaining, G_remaining, h_remaining, L_np, U_np, y0 = np.array([0.5]*num_var))
start_time = time.time()
y_r, info_r = nlp_r.solve(np.array([0.05]*num_var))
# y_r, info_r = nlp_r.solve(y.detach().numpy())
print(time.time() - start_time)
print("total time: ", time.time() - start_time + inference_time)
info_r['obj_val']


0.07234883308410645
total time:  0.07410669326782227


/tmp/ipykernel_3078455/560160664.py:41: FutureWarning: The class named 'problem' will soon be deprecated in CyIpopt. Please replace all uses and use 'Problem' going forward.
  nlp = ipopt.problem(n=num_var,


-39.8476382080661

In [39]:
y = y_r
res = []
for i in range(num_ineq):
    res.append(0.5 * y.T @ H_np[i] @ y + G_np[i].T @ y - h_np[i])
res = np.array(res)
print(res.max())
print((L_np - y).max())
print((y - U_np).max())

9.966799691119022e-09
4.998601443872985e-08
4.9963584380918746e-08


In [35]:
set(inactive_ineq).intersection(set(active_ineq))

set()